In [1]:
import pandas as pd
# Load the dataset
df = pd.read_csv('cleaned_logistics_data.csv')

In [2]:
# 1. Load the selected features data
selected_features_df = pd.read_csv('X_ridge_selected.csv')  # Replace with your feature selection file path
target_variable = 'is_cutoff'  # Replace with your target variable name if different

In [3]:
# Assuming 'df' is your original DataFrame with the target variable
y = df[target_variable]  


In [4]:
from sklearn.model_selection import train_test_split
# 4. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    selected_features_df, y, test_size=0.2, random_state=42
)  # Adjust test_size and random_state as needed


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential([
    Dense(512, activation='relu', input_shape=(3016,), 
           kernel_regularizer=l2(0.001),  # L2 regularization
           kernel_initializer='he_normal'),  # Better weight initialization
    BatchNormalization(),
    Dropout(0.4),  # Increased dropout rate
    
    Dense(256, activation='relu', 
          kernel_regularizer=l2(0.001),
          kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu', 
          kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Adjust based on your output
])

# Compile with advanced optimizer settings
optimizer = Adam(learning_rate=0.0005,  # Reduced learning rate
                 beta_1=0.9,
                 beta_2=0.999,
                 epsilon=1e-07)

model.compile(optimizer=optimizer, 
              loss='mean_squared_error',
              metrics=['accuracy'])

# Advanced training callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=15,  # Increased patience
    restore_best_weights=True,
    min_delta=0.0001
)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce learning rate
    patience=5,
    min_lr=1e-6
)

# Training with more epochs and callbacks
history = model.fit(
    X_train, y_train,
    epochs=100,  # Increased number of epochs
    batch_size=32,  # Experiment with batch size
    validation_split=0.2,
    callbacks=[early_stopping, lr_reducer]
)

Epoch 1/100
2898/2898 [==============================] - 55s 19ms/step - loss: 0.5863 - accuracy: 0.8235 - val_loss: 0.2527 - val_accuracy: 0.8453 - lr: 5.0000e-04
Epoch 2/100
2898/2898 [==============================] - 50s 17ms/step - loss: 0.1775 - accuracy: 0.8415 - val_loss: 0.1481 - val_accuracy: 0.8368 - lr: 5.0000e-04
Epoch 3/100
2898/2898 [==============================] - 51s 18ms/step - loss: 0.1430 - accuracy: 0.8417 - val_loss: 0.1362 - val_accuracy: 0.8471 - lr: 5.0000e-04
Epoch 4/100
2898/2898 [==============================] - 51s 18ms/step - loss: 0.1373 - accuracy: 0.8432 - val_loss: 0.1305 - val_accuracy: 0.8476 - lr: 5.0000e-04
Epoch 5/100
2898/2898 [==============================] - 52s 18ms/step - loss: 0.1339 - accuracy: 0.8451 - val_loss: 0.1269 - val_accuracy: 0.8525 - lr: 5.0000e-04
Epoch 6/100
2898/2898 [==============================] - 53s 18ms/step - loss: 0.1302 - accuracy: 0.8464 - val_loss: 0.1241 - val_accuracy: 0.8517 - lr: 5.0000e-04
Epoch 7/100
2898

Epoch 51/100
2898/2898 [==============================] - 59s 20ms/step - loss: 0.0844 - accuracy: 0.9033 - val_loss: 0.0771 - val_accuracy: 0.9135 - lr: 1.2500e-04
Epoch 52/100
2898/2898 [==============================] - 58s 20ms/step - loss: 0.0846 - accuracy: 0.9026 - val_loss: 0.0775 - val_accuracy: 0.9112 - lr: 1.2500e-04
Epoch 53/100
2898/2898 [==============================] - 58s 20ms/step - loss: 0.0829 - accuracy: 0.9056 - val_loss: 0.0739 - val_accuracy: 0.9165 - lr: 1.2500e-04
Epoch 54/100
2898/2898 [==============================] - 57s 20ms/step - loss: 0.0826 - accuracy: 0.9057 - val_loss: 0.0761 - val_accuracy: 0.9145 - lr: 1.2500e-04
Epoch 55/100
2898/2898 [==============================] - 57s 20ms/step - loss: 0.0825 - accuracy: 0.9056 - val_loss: 0.0751 - val_accuracy: 0.9157 - lr: 1.2500e-04
Epoch 56/100
2898/2898 [==============================] - 57s 20ms/step - loss: 0.0820 - accuracy: 0.9065 - val_loss: 0.0798 - val_accuracy: 0.9100 - lr: 1.2500e-04
Epoch 57/1

In [6]:
model.get_config()

{'name': 'sequential',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 3016),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 3016),
    'units': 512,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'HeNormal',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': {'module': 'keras.regularizers',
     'class_name': 'L2',
     'config': {'l2': 0.0010000000474974513},
     'registered_name': None},
    'bias_regularizer': None,
    'activity_regul

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1544704   
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0